In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
import plotly_express as px

# Objectif

Le Nutriscore est une information qui peut être utile pour comparer divers produits d'une même catégorie les uns avec les autres. Cependant, tous les produits ne disposent pas de cette information et il pourrait être intéressant de pouvoir calculer ce score à partir des indications disponibles sur les étiquettes des produits (ou même simplement en scannant le code-barre).

Nous allons donc essayer de calculer le nutriscore ou le nutrigrade en nous basant sur les données disponibles sur l'étiquette d'un produit.

#### Éléments défavorables au score

- Apport calorique pour cent grammes.
- Teneur en sucre.
- Teneur en graisses saturées.
- Teneur en sel.

#### Éléments favorables au score

- Teneur en fruits, légumes, légumineuses (dont les légumes secs), oléagineux, huiles de colza, de noix et d'olive.
- Teneur en fibres.
- Teneur en protéines.

Pour calculer la teneur de fruits et légumes, les féculents (tel que pomme de terre, patate douce, taro, manioc et tapioca) ne sont pas pris en compte.

Pour les fromages, la teneur en protéines est toujours prise en compte car celle-ci est liée à celle en calcium. Ceci améliore le nutri-score des fromages et la cohérence entre celui-ci et les recommandations nutritionnelles du Haut Conseil de la Santé Publique. Celles-ci recommandent en effet de consommer des produits laitiers plusieurs fois par jour. 

# 2. Présentation générale du jeu de données <a class="anchor" id="P02"></a>

Le [jeu de données](https://static.openfoodfacts.org/data/en.openfoodfacts.org.products.csv) utilisé dans le cadre de l'appel à projets de l'agence *Santé publique France* qui consiste à rendre les données de santé plus accessibles, est une liste de 2.251.894 produits alimentaires répertoriés par les volontaires de l'association [Open Food Facts](https://world.openfoodfacts.org).

Chacun des produits référencé est décrit par un certain nombre de caractèristiques nutritionels *(taux de graisse, de sucre, de sel, de fibres, de protéines, de vitamines, etc.)* et par des méta-données *(code-barre, nom du produit, catégorie, lieu de production, data d'ajout dans la DB, auteur de l'ajout, etc.)*.

> Le jeu de données complet fait plus de 5.9GB. Et il est difficile de travailler avec un tel volume de donnée, donc nous avons crée un jeu de données réduit (dans Cleaning_01.ipynb) que nous allons utiliser ici.

In [2]:
filename = 'data/data_low_cols.csv'

In [42]:
data = pd.read_csv(filename, sep=',', dtype={'code':'str'})

# Explorons le jeu de données reduit

In [43]:
data.head(3)

,code,url,product_name,quantity,categories_tags,labels_tags,ingredients_tags,allergens,traces_tags,serving_size,...,vitamin-a_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,potassium_100g,calcium_100g,iron_100g,fruits-vegetables-nuts-estimate-from-ingredients_100g,nutrition-score-fr_100g
0,00000000000000225,http://world-en.openfoodfacts.org/product/0000...,jeunes pousses,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00000000000003429145,http://world-en.openfoodfacts.org/product/0000...,L.casei,NaN,NaN,NaN,"en:semi-skimmed-milk,en:dairy,en:milk,es:azuca...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,NaN
2,0000000000017,http://world-en.openfoodfacts.org/product/0000...,Vitória crackers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
data.shape

(2251894, 48)

In [45]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [46]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
serving_quantity,526776.000,21092745435793836.000,15308918127999203328.000,0.000,28.000,55.000,114.000,11111111111111100858368.000
additives_n,755169.000,2.013,2.880,0.000,0.000,1.000,3.000,49.000
nutriscore_score,776383.000,9.091,8.849,-15.000,1.000,10.000,16.000,40.000
nova_group,679561.000,3.382,0.992,1.000,3.000,4.000,4.000,4.000
ecoscore_score,516814.000,43.546,25.704,-30.000,27.000,39.000,65.000,125.000
energy-kj_100g,162957.000,40903789888675473637899005574841892864.000,16512014422283680237661674859037450043392.000,0.000,394.000,980.000,1619.000,6665558888888888950360610417759390841962496.000
energy-kcal_100g,1737339.000,62672267.876,76154079787.096,0.000,100.000,259.000,400.000,100000000376832.000
energy_100g,1786722.000,3730607721228534090965745911682039808.000,4986640698625920150338062244302641168384.000,0.000,418.000,1079.000,1674.000,6665558888888888950360610417759390841962496.000
fat_100g,1777186.000,562754208033.989,750124863814886.500,0.000,0.800,7.000,21.200,1000000000000000000.000
saturated-fat_100g,1731439.000,57722.281,75945867.241,0.000,0.100,1.800,7.000,99932728111.000


> On remarque que beaucoup de colonnes numériques ont des valeurs min, max (et donc std) clairement hors normes.
>
> Il va falloir s'occuper de toutes ces valeurs aberrantes.

In [47]:
data.describe(exclude="number").T

,count,unique,top,freq
code,2251894,2251873,3477610001135,2
url,2251894,2251876,http://world-en.openfoodfacts.org/product/3477...,2
product_name,2166180,1387469,Miel,1449
quantity,576236,39373,500 g,24211
categories_tags,1011088,79235,en:snacks,34325
labels_tags,462090,64122,en:organic,47495
ingredients_tags,753418,569887,"en:extra-virgin-olive-oil,en:oil-and-fat,en:ve...",1391
allergens,199182,7300,en:milk,42134
traces_tags,136399,13774,en:nuts,13203
serving_size,530697,47576,100g,24366


> On remarque qu'il y a au moins un doublon sur les codes, il faudra donc le(s) supprimer.

#### Affichons les taux de valeurs manquantes pour chaque colonne.

In [48]:
# Définissons une fonction qui nous permet d'afficher 
# facilement le nombre de valeurs manquantes / présentes et leur taux

def print_fill_rate(dataset, col_array):
    fill_count = dataset[col_array].notnull().sum()
    fill_ratio = fill_count/dataset.shape[0]*100.0
    total = dataset.shape[0]
    
    for k, v in zip(fill_count.keys(), fill_count):
        fraction = v/dataset.shape[0]*100.0
        print(f"{k.rjust(53)} | {total-v:8} lignes vides | replissage: {fraction:6.2f}%")
        
    return fill_ratio

In [49]:
fill_ratio = print_fill_rate(data, data.columns)

                                                 code |        0 lignes vides | replissage: 100.00%
                                                  url |        0 lignes vides | replissage: 100.00%
                                         product_name |    85714 lignes vides | replissage:  96.19%
                                             quantity |  1675658 lignes vides | replissage:  25.59%
                                      categories_tags |  1240806 lignes vides | replissage:  44.90%
                                          labels_tags |  1789804 lignes vides | replissage:  20.52%
                                     ingredients_tags |  1498476 lignes vides | replissage:  33.46%
                                            allergens |  2052712 lignes vides | replissage:   8.85%
                                          traces_tags |  2115495 lignes vides | replissage:   6.06%
                                         serving_size |  1721197 lignes vides | replissage:  23.57%


#### Affichons ces taux sous forme de graphique interactif

In [50]:
fill_ratio_df = pd.DataFrame(fill_ratio, columns=["fill_rate"])

fig = px.bar(fill_ratio_df, y="fill_rate", 
             #width=900,
             height=600,
             color='fill_rate', 
             title="Taux de valeurs présentes par colonnes",
             labels={
                "fill_rate": "Remplissage (en %)",
                "index": "",
                },
            )
fig.update_coloraxes(showscale=False)
fig.update_xaxes(tickangle = -45)
fig.show()

> Visiblement, même allégé des colonnes les plus vides, ce jeu de données présente encore beaucoup de trous.

## Supprimons les doublons

In [52]:
data_duplicate = data.copy()

#### Commençons par supprimer les produits en double sur la base du code-barre

In [53]:
duplicated = data_duplicate.code.duplicated(keep='first')
duplicated.sum()

21

In [54]:
data_duplicate.drop_duplicates(subset=['code'], inplace=True) 

> Nous avons donc supprimé 21 produits en double.

#### Ensuite vérifions si il y a des lignes similaires en excluant le code-barre

In [55]:
cols_without_code = list(data_duplicate.columns)
cols_without_code.remove("code")

In [56]:
duplicated = data_duplicate[cols_without_code].duplicated(keep='first')
duplicated.sum()

0

> Il n'y a visiblement pasd'autres doublons.

#### Reportons les modifications sur le data_clean

In [57]:
data_clean = data_duplicate.copy()
data_clean.shape

(2251873, 48)

## Supprimons les valeurs aberrantes

In [58]:
data_clean.describe().T

,count,mean,std,min,25%,50%,75%,max
serving_quantity,526772.000,21092905601827232.000,15308976251396298752.000,0.000,28.000,55.000,114.000,11111111111111100858368.000
additives_n,755157.000,2.013,2.880,0.000,0.000,1.000,3.000,49.000
nutriscore_score,776372.000,9.091,8.848,-15.000,1.000,10.000,16.000,40.000
nova_group,679549.000,3.382,0.992,1.000,3.000,4.000,4.000,4.000
ecoscore_score,516799.000,43.545,25.704,-30.000,27.000,39.000,65.000,125.000
energy-kj_100g,162948.000,40906049100871989863344482413210238976.000,16512470414618234403468769868866868215808.000,0.000,394.000,980.000,1619.000,6665558888888888950360610417759390841962496.000
energy-kcal_100g,1737326.000,62672736.834,76154364708.028,0.000,100.000,259.000,400.000,100000000376832.000
energy_100g,1786708.000,3730636952926213338818390095279685632.000,4986660235347371010035389652996064280576.000,0.000,418.000,1079.000,1674.000,6665558888888888950360610417759390841962496.000
fat_100g,1777171.000,562758957893.806,750128029477957.875,0.000,0.800,7.000,21.200,1000000000000000000.000
saturated-fat_100g,1731424.000,57722.781,75946196.215,0.000,0.100,1.800,7.000,99932728111.000


Comme on peut le constater, un certain nombre de colonnes ont des valeurs étonnantes...

> `serving_quantity`
> 
> le maximum est bien au delà du troisième quartile, et l'écart type est énorme...

> Les colonnes `*_100g`
>
> le maximum dépasse les 100g et ce n'est pas possible *(car d'après la notice **"fields that end with _100g correspond to the amount of a nutriment (in g, or kJ for energy) for 100 g or 100 ml of product"** et donc il ne peut y avoir de valeurs > 100)*
>
>> Pour `cholesterol_100g` le maximum semble être à **3.1g** (3100mg) pour 100g /// Cervelle de veau cuite
>
>> Pour `fat_100g` le maximum semble être à **100g** pour 100g /// Huile d'avocat
>
>> Pour `saturated-fat_100g` le maximum semble être à **92.6** pour 100g /// Pain de friture (pas l'huile ?)
>
>> Pour `proteins_100g` le maximum semble être à **87.6g** pour 100g /// Gélatine alimentaire
>
>> Pour `sugars_100g` le maximum semble être à **99.8g** pour 100g /// Fructose
>
>> Pour `carbohydrates_100g` le maximum semble être à **99.8g** pour 100g /// Fructose
>
>> Pour `sodium_100g` le maximum semble être à **39.1g** (39100 mg)pour 100g /// sel blanc non iodé non fluoré 
>
>> Pour `salt_100g` le maximum semble être à **39.1g** (39100 mg)pour 100g /// sel blanc non iodé non fluoré 
>
>> Pour `iron_100g` le maximum semble être à **0,0064g** (6.4 mg) pour 100g /// Boudin noir
>
>> Pour `calcium_100g` le maximum semble être à **2g** (2000 mg) pour 100g /// Meloukhia en poudre 
>
>> Pour `fiber_100g` le maximum semble être à **43.5g** pour 100g /// Cannelle
>
>> Pour `energy-kcal_100g` le maximum semble être à **900 kcal** pour 100g /// Huile d'avocat
>
>> Pour `energy-kj_100g` le maximum semble être à **3765.6 kcal** pour 100g /// Huile d'avocat

> `carbohydrates_100g`, `sugars_100g`, `fiber_100g`, `proteins_100g`, `nutriscore_score`, `nutrition-score-fr_100g`, `ecoscore_score`
>
> le minimum est sous 0, ce qui ne semble pas cohérent avec l'intitulé de ces colonnes.

## TODO

> mixer la suppressions sur les valeurs réelles quand elles sont connues et les valeurs IQR dans le cas contraire  ?

le taux de sel ~= 2.54 * taux de sodium

donc normalement salt_100g = 2.54 * sodium_100g


#### Éléments défavorables au score

- Apport calorique pour cent grammes. **->** `energy-kcal_100g` ou `energy-kj_100g` ou `energy_100g`
- Teneur en sucre. **->** `sugars_100g` ou `carbohydrates_100g`
- Teneur en graisses saturées. **->** `saturated-fat_100g`
- Teneur en sel. **->** `salt_100g` *(ou `sodium_100g` car 40% de sodium dans le sel + 60% de chlorure)*

> `energy_100g`, `sugars_100g`, `saturated-fat_100g`, `salt_100g`

#### Éléments favorables au score

- Teneur en fruits, légumes, légumineuses (dont les légumes secs), oléagineux, huiles de colza, de noix et d'olive. **->** `fruits-vegetables-nuts-estimate-from-ingredients_100g` **?** ou `fruits-vegetables-nuts_100g`
- Teneur en fibres. **->** `fiber_100g`
- Teneur en protéines. **->** `proteins_100g`

> `fruits-vegetables-nuts-estimate-from-ingredients_100g`, `fiber_100g`, `proteins_100g`

# Notes & Essais

In [ ]:
data.states

In [ ]:
data.states.iat[0]

In [ ]:
data.states_fr